In [1]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
import torch

In [6]:
!pip install pyarrow

Traceback (most recent call last):
  File "/usr/bin/pip", line 5, in <module>
    from pkg_resources import load_entry_point
  File "/apps/free76/jupyter.py/3.7/lib/python3.7/site-packages/pkg_resources/__init__.py", line 124
    f"{v} is an invalid version and will not be supported in "
                                                             ^
SyntaxError: invalid syntax


In [2]:
import pyarrow

ModuleNotFoundError: No module named 'pyarrow'

In [3]:
train_feather_path = "../../data/train_test_splits/cog_train_family_tax_level.feather"
test_feather_path = "../../data/train_test_splits/cog_test_family_tax_level.feather"

train_df = pd.read_feather(train_feather_path)
val_df = pd.read_feather(args_dict[test_feather_path])

ImportError: pyarrow is not installed

you can install via conda
conda install pyarrow -c conda-forge
or via pip
pip install -U pyarrow


In [ ]:
tsne_bin = TSNE(n_components=2, random_state=4, perplexity=50, learning_rate=100, max_iter=5000)
bin_embeds_2d = tsne_bin.fit_transform(binary_mlm_embeddings)